In [ ]:
import numpy as np
import torch
from torchsummary import summary

In [ ]:
vgg16model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)

In [ ]:
print(vgg16model)

In [ ]:
def stretch_image(x):
    img = x.detach().numpy().squeeze()
    img = np.transpose(img, (1, 2, 0))
    img  = (img - img.min()) / (img.max() - img.min()) * 255.
    return img.astype(np.uint8)

In [ ]:
from torch.autograd import Variable
from tqdm import tqdm, trange

In [ ]:
nchannels = 64

images = []

for channel in range(nchannels):
    with torch.enable_grad():
        random_tensor = torch.randn((3, 224, 224), requires_grad=True)
        random_tensor = random_tensor.unsqueeze(0)

    lr = 1.0
    x = Variable(random_tensor, requires_grad=True)
    optimizer = torch.optim.Adam([x], lr=lr, weight_decay=1e-6)

    nepochs = 25
    prog_bar = trange(nepochs, desc='Loss:', leave=True)
    for i in prog_bar:
        optimizer.zero_grad()
        output = x.clone()
        for layer in range(17):
            output = vgg16model.features[layer](output)
    
        loss = -torch.mean(output[:, channel, :, :])
        prog_bar.set_description(f'Loss: {loss} Channel: {channel}')
        prog_bar.refresh()
        loss.backward()
        optimizer.step()
        
    images.append(stretch_image(x))

In [ ]:
import ipyplot
ipyplot.plot_images(images, max_images=nchannels, img_width=200)